# Initialisation

> Setting up the device to synchronise systems

In [ ]:
#| default_exp device

In [ ]:
#| hide
from nbdev.showdoc import *

# Import required libraries
import serial
import serial.tools.list_ports
import time

The software drivers provide a simple interface to communicate with the device in order to send event signals from behavioural tasks to synchronise with recorded data.

## Importing the drivers
Once installed, the drivers can be imported into the task script like any package.

In [ ]:
# Import driver package
import syncmaster

All contents of the driver package can then be accessed using the prefix `syncmaster.`, i.e. the `SyncMaster` object is accessed using `syncmaster.SyncMaster()`.

Alternatively, all contents of the package can be imported directly into the present script's namespace:

In [ ]:
# Import all package contents into current namespace
from syncmaster import *

Package contents can then be accessed using their names directly without the need for a prefix, i.e. `SyncMaster()`.

While slightly more convenient, this runs the risk of colliding with function and object names imported from other packages, so we recommend using the package prefix where possible.

## Creating the device object
The software communicates with the device hardware using a software object. All device commands are controlled using this object.

In [ ]:
#| export
'''
    SyncMaster device drivers

    Oxford Neural Interfacing
    Written by Conor Keogh
    conor.keogh@nds.ox.ac.uk
    16/04/2023

    Provides functions for interacting with SyncMaster device
'''

'''
SyncMaster device class
Provide functions to interact with device
Finds device and confirms presence
Sends messages to trigger device
'''
import serial
import serial.tools.list_ports
import time

class SyncMaster:

    def __init__(self):
        '''
        Device object for controlling synchronisation
        '''

        '''
        Define constants for device configuration
        '''
        # Define messages
        self.GREETING = b'<best wishes>'
        self.RESPONSE = b'warmest regards'
        self.startMarker = '<'
        self.endMarker = '>'
    
        # Define trigger pulse widths (val*10, milliseconds)
        self.STARTPULSE = 5 # 50ms
        self.ENDPULSE = 10 # 100ms
        self.EVENT1PULSE = 15 # 150ms
        self.EVENT2PULSE = 20 # 200ms
    
        # Define COM port settings
        self.BAUDRATE = 115200
        
        # Get all serial ports
        ports = serial.tools.list_ports.comports()

        # For each port: try accessing and checking for acknowledge message
        port_found = False
        for port in ports:
            try:
                # Connect to serial port
                self.ser = serial.Serial(port.device, self.BAUDRATE, timeout=1, write_timeout=1)

                # Send test message and read response; repeat 3 times and keep third
                for _ in range(3):
                    self.sendMessage(self.HOST_MESSAGE)
                    response = self.ser.readline()

                # Check if response is appropriate
                if response.decode().strip() == self.RESPONSE:
                    self.target_port = port.device
                    port_found = True

                # Close port
                self.ser.close()

            except Exception as e:
                # Do nothing - just ignore failed ports
                pass

        # If port found: connect to port
        if port_found:
            self.ser = serial.Serial(self.target_port, self.BAUDRATE, timeout=5)

        # If port not found: raise error
        else:
            raise Exception("Device not found")
#            print("Device not found")

    ''' Send required messages over serial '''

    def start(self):
        ''' Send start signal '''
        self.sendMessage(self.STARTMARKER)

    def end(self):
        ''' Send end signal '''
        self.sendMessage(self.ENDMARKER)

    def event1(self):
        ''' Send event 1 signal '''
        self.sendMessage(self.EVENT1)

    def event2(self):
        ''' Send event 2 signal '''
        self.sendMessage(self.EVENT2)
        
    def event(self, eventID):
        ''' Create event marker '''
        self.sendMessage(eventID)

    # Send message via serial port
    def sendMessage(self, message):
        '''
        Send message over serial port
        Takes message to send
        '''
        message_prepared = self.startMarker + str(message) + self.endMarker
        self.ser.write(message_prepared.encode())

    # Close channel
    def close(self):
        '''
        Closes device connection
        '''
        self.ser.close()
        
    # Send test signal
    def testSignal(self):
        '''
        Sends test pulses over output port once per second for five seconds
        '''
        for _ in range(5):
            self.start()
            time.sleep(1)

In order to use the device in a task script, we must first create the device object. This should be done once at the beginning of the script.

In [ ]:
#|eval: false
# Create device object
device = syncmaster.SyncMaster()

This automatically carries out all initialisation procedures, including locating the device on the host system and ensuring the device is communicating correctly.

Note that an error will be produced on attempting to create the device object if the device is not connected to the host system.

Once the device has been initialised once in this way, it is ready to send event signals to the recording system as outlined in the `triggering` section.

## Shutting down the device
On completing the task, the communicating channel between the host system and the device should be shut down.

In [ ]:
show_doc(SyncMaster.close)

The device should be shut down using the `close` command at the end of the task. 

In [ ]:
#|eval: false
# Close device
device.close()

It can then be disconnected from the host system.

This ensures that all communications ports are closed correctly to avoid any errors.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()